<center><img src="./logo.png" alt="Header" style="width: 800px;"/></center>

In [1]:
#!/usr/bin/env python3
# encoding: utf-8
#导入相关的模块 Import related modules
import mediapipe as mp
import threading
import cv2 as cv
import time
import math
from time import sleep
import ipywidgets.widgets as widgets
image_widget = widgets.Image(format='jpeg', width=640, height=480)  #设置摄像头显示组件 Set up the camera display component

In [2]:
# 将BGR图像转换为JPEG格式的字节流 Convert a BGR image to a JPEG byte stream
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv.imencode('.jpg', value)[1])

# 主函数 Main function

In [3]:
class FaceDetector:
    def __init__(self, minDetectionCon=0.5):
        self.mpFaceDetection = mp.solutions.face_detection
        self.mpDraw = mp.solutions.drawing_utils
        self.facedetection = self.mpFaceDetection.FaceDetection(min_detection_confidence=minDetectionCon)

    def findFaces(self, frame):
        img_RGB = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        self.results = self.facedetection.process(img_RGB)
        bboxs = []
        if self.results.detections:
            for id, detection in enumerate(self.results.detections):
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, ic = frame.shape
                bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                       int(bboxC.width * iw), int(bboxC.height * ih)
                bboxs.append([id, bbox, detection.score])
                frame = self.fancyDraw(frame, bbox)
                cv.putText(frame, f'{int(detection.score[0] * 100)}%',
                           (bbox[0], bbox[1] - 20), cv.FONT_HERSHEY_PLAIN,
                           3, (255, 0, 255), 2)
        return frame, bboxs

    def fancyDraw(self, frame, bbox, l=30, t=10):
        x, y, w, h = bbox
        x1, y1 = x + w, y + h
        cv.rectangle(frame, (x, y),(x + w, y + h), (255, 0, 255), 2)
        # Top left x,y
        cv.line(frame, (x, y), (x + l, y), (255, 0, 255), t)
        cv.line(frame, (x, y), (x, y + l), (255, 0, 255), t)
        # Top right x1,y
        cv.line(frame, (x1, y), (x1 - l, y), (255, 0, 255), t)
        cv.line(frame, (x1, y), (x1, y + l), (255, 0, 255), t)
        # Bottom left x1,y1
        cv.line(frame, (x, y1), (x + l, y1), (255, 0, 255), t)
        cv.line(frame, (x, y1), (x, y1 - l), (255, 0, 255), t)
        # Bottom right x1,y1
        cv.line(frame, (x1, y1), (x1 - l, y1), (255, 0, 255), t)
        cv.line(frame, (x1, y1), (x1, y1 - l), (255, 0, 255), t)
        return frame


# 主进程 Main Process

In [4]:
if __name__ == '__main__':
    capture = cv.VideoCapture(0)
    # capture.set(0, cv.VideoWriter.fourcc('M', 'J', 'P', 'G'))
    capture.set(cv.CAP_PROP_FRAME_WIDTH, 320)
    capture.set(cv.CAP_PROP_FRAME_HEIGHT, 240)
    print("capture get FPS : ", capture.get(cv.CAP_PROP_FPS))
    pTime, cTime = 0, 0
    face_detector = FaceDetector(0.75)
    display(image_widget) 
    try:
        while capture.isOpened():
            ret, frame = capture.read()
            # frame = cv.flip(frame, 1)
            frame,_ = face_detector.findFaces(frame)
            if cv.waitKey(1) & 0xFF == ord('q'): break
            cTime = time.time()
            fps = 1 / (cTime - pTime)
            pTime = cTime
            text = "FPS : " + str(int(fps))
            cv.putText(frame, f"FPS: {fps:.1f}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            image_widget.value = bgr8_to_jpeg(frame)
            # cv.imshow('frame', frame)
    except KeyboardInterrupt:
        capture.release()
        # cv.destroyAllWindows()

capture get FPS :  15.0


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1722592147.242512  867797 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Image(value=b'', format='jpeg', height='480', width='640')

In [6]:
#使用完成对象记住释放掉对象，不然下一个程序使用这个对象模块会被占用，导致无法使用
#Release resources
capture.release()            